# Calculating Forest Loss in Protected Areas

In [1]:
import os
import ee
import geemap 
import pandas as pd

#ee.Authenticate()
ee.Initialize()

In [2]:
# Define the bounding box of Colombia (using a shapefile uploaded to the assets folder)
colombiaMpios = ee.FeatureCollection('projects/ee-juamiji/assets/muni_runapf')

# Import the Forest loss image
flossHansen = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")

# Import the Primary Tropical Forest cover image collection and filter it for Colombia
primaryForest_ideam = ee.Image("projects/ee-juamiji/assets/SouthAmerica_2001_primary").selfMask()
primaryForest_ideam_COL = primaryForest_ideam.clip(colombiaMpios)

# Clip the Forest loss image to Colombia's boundaries
flossHansen_COL = flossHansen.clip(colombiaMpios)


c:\Users\juami\anaconda3\envs\geo\lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for UMD/hansen/global_forest_change_2023_v1_11! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2023_v1_11

  warnings.warn(warning, category=DeprecationWarning)


In [3]:
# Visualization parameters for the map
viz_params = {
    'min':1,  # Minimum value for visualization
    'max': 1,  # Maximum value for visualization
    'palette': ['008000']  # Color gradient
}

# Initialize the map using geemap
Map = geemap.Map(center=[4.0, -72.0], zoom=5)  # Center roughly in Colombia with an appropriate zoom level
Map.addLayer(primaryForest_ideam_COL, viz_params, "Primary Tropical Forests")
Map

Map(center=[4.0, -72.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [4]:
# Create a mask where fcover is equal to 1.
primaryMask = primaryForest_ideam_COL.eq(1)

# Masking the floss imagery using the primary mask (cover equals 1)
flossHansen_COLmasked = flossHansen_COL.updateMask(primaryMask)

# Select the 'lossyear' band from the masked forest loss image
loss_year = flossHansen_COLmasked.select(['lossyear'])

## Code for all years from 2000 to 2023:

In [5]:
# Calculate the total loss area per municipality
def calculate_loss_area(feature, year):

    loss_year_mask = loss_year.eq(year)

    # Calculate the loss area for the current year in square meters
    loss_area_image = loss_year_mask.multiply(ee.Image.pixelArea())

    # Use reduceRegion to calculate the sum of the loss area within the feature's geometry
    loss_area = loss_area_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e9
    ).get('lossyear')  # Replace 'lossyear' with the correct property name if different
    
     # Dynamically name the loss area property
    loss_area_property_name = f'lossarea{year}'

    # Return a new feature with only the 'lossArea' and 'codmpio' properties
    return ee.Feature(None, {
        loss_area_property_name: loss_area,
        'codmpio': feature.get('codmpio'),  # Ensure 'codmpio' exists in the original feature
    })

# Dictionary to store results for each year
year_results_dict = {}

for year in range(0, 24):
    # Use a lambda function to pass the current year into the map function
    current_year_results = colombiaMpios.map(lambda feature: calculate_loss_area(feature, year))
    year_results_dict[f'year_results{year}'] = current_year_results
    # Optionally print the first feature's info to check
    print(year_results_dict[f'year_results{year}'].first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'lossarea0': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'lossarea1': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'lossarea2': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'lossarea3': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'lossarea4': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'lossarea5': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'lossarea6': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'lossarea7': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'lossarea8': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000202', 'properties': {'lossarea9': 0}}


In [6]:
# Define a function to export FeatureCollections to Google Drive
def export_to_drive(feature_collection, year, description_prefix="ForestLossIllegalmData", folder_name="EarthEngineExports"):
    # Create the export task
    export_task = ee.batch.Export.table.toDrive(
        collection=feature_collection,
        description=f"{description_prefix}_Year{year}",
        folder=folder_name,
        fileNamePrefix=f"ForestLoss_Illegal_Year{year}",
        fileFormat="CSV"
    )
    # Start the export task
    export_task.start()
    print(f"Exporting {description_prefix}_Year{year} to Google Drive...")

# Loop through each item in the dictionary and initiate an export
for year, feature_collection in year_results_dict.items():
    # Extract the year from the key (which is a string like 'year_results11')
    extracted_year = int(year.replace('year_results', ''))
    export_to_drive(feature_collection, extracted_year)

Exporting ForestLossIllegalmData_Year0 to Google Drive...
Exporting ForestLossIllegalmData_Year1 to Google Drive...
Exporting ForestLossIllegalmData_Year2 to Google Drive...
Exporting ForestLossIllegalmData_Year3 to Google Drive...
Exporting ForestLossIllegalmData_Year4 to Google Drive...
Exporting ForestLossIllegalmData_Year5 to Google Drive...
Exporting ForestLossIllegalmData_Year6 to Google Drive...
Exporting ForestLossIllegalmData_Year7 to Google Drive...
Exporting ForestLossIllegalmData_Year8 to Google Drive...
Exporting ForestLossIllegalmData_Year9 to Google Drive...
Exporting ForestLossIllegalmData_Year10 to Google Drive...
Exporting ForestLossIllegalmData_Year11 to Google Drive...
Exporting ForestLossIllegalmData_Year12 to Google Drive...
Exporting ForestLossIllegalmData_Year13 to Google Drive...
Exporting ForestLossIllegalmData_Year14 to Google Drive...
Exporting ForestLossIllegalmData_Year15 to Google Drive...
Exporting ForestLossIllegalmData_Year16 to Google Drive...
Exporti